<a href="https://colab.research.google.com/github/dcolinmorgan/MYC/blob/master/LTCOPD_sex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import s3fs
import pandas as pd
import numpy as np
from psutil import *
import matplotlib.pyplot as plt

In [0]:
!pip install cupy-cuda101==7.4.0
import cupy as cp
# print(cp.__version__)
!pip freeze | grep cupy

In [0]:
# !git clone https://github.com/netZoo/netZooPy.git
!git clone --single-branch --branch devel https://github.com/netZoo/netZooPy.git
os.chdir('netZooPy')
!pip install -e ./

In [0]:
import netZooPy
from netZooPy.panda import Panda
from netZooPy.lioness import Lioness
os.chdir('..')

In [0]:
!pip install GEOparse

In [0]:
import GEOparse
gse = GEOparse.get_GEO(geo="GSE76925", destdir="./")

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# illumina_labels=pd.read_csv('drive/My Drive/Colab Notebooks/illumina_labels.txt',sep=';',header=0)
illumina_labels=pd.read_csv('drive/My Drive/Colab Notebooks/GPL10558_illumina_prb_loc.txt',sep='\t',index_col=0,header=None,names=['loc','symbol'])

In [0]:
gpl = gse.gpls[next(iter(gse.gpls))]
data = gse.pivot_samples(values="VALUE")#, gpl=gpl, annotation_column="Symbol")
data.sort_index

In [0]:
pheno=gse.phenotype_data.rename(columns={'characteristics_ch1.0.age': 'age',"characteristics_ch1.1.Sex":"sex","characteristics_ch1.3.bmi":"bmi","characteristics_ch1.6.fev1.pp":"fev1","characteristics_ch1.4.packyears":"PY"})
pheno[['age',"sex","bmi","fev1","PY"]].to_csv('pheno_data.txt',sep='\t',index=True,header=True)

In [0]:
data2=data.merge(illumina_labels['symbol'],left_index=True,right_index=True)
data2.index=data2['symbol']
del data2['symbol']
data2.columns=[gse.phenotype_data['characteristics_ch1.1.Sex']+gse.phenotype_data['characteristics_ch1.5.copd']]
data2.loc[:,'Mcase'].to_csv('Mcase.txt',sep='\t',index=True,header=False)
data2.loc[:,'Mcont'].to_csv('Mcont.txt',sep='\t',index=True,header=False)

data3=data.merge(illumina_labels,left_index=True,right_index=True)
data3.index=data3['symbol']
data3=data3[data3['loc']!='Y']
del data3['symbol'], data3['loc']
data3.columns=[gse.phenotype_data['characteristics_ch1.1.Sex']+gse.phenotype_data['characteristics_ch1.5.copd']]
data3.loc[:,'Fcase'].to_csv('Fcase.txt',sep='\t',index=True,header=False)
data3.loc[:,'Fcont'].to_csv('Fcont.txt',sep='\t',index=True,header=False)

In [0]:
LCL_ppi=pd.read_csv('https://granddb.s3.amazonaws.com/optPANDA/ppi/ppi_complete.txt',sep='\t',header=None)
LCL_ppi.to_csv('ppi_complete.txt',sep='\t',index=False,header=False)

coding_LCL_motif=pd.read_csv('https://granddb.s3.amazonaws.com/optPANDA/motifs/Hugo_motifCellLine.txt',sep='\t',header=None)
coding_LCL_motif.to_csv('coding_LCL_motif.txt',sep='\t',index=False,header=False)

motif_data='coding_LCL_motif.txt'
ppi_data='ppi_complete.txt'

In [0]:
# panda_obj1 = Panda('Fcase.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = True, remove_missing=True, keep_expression_matrix = False,modeProcess = 'intersection')
# np.save('Panda_Fcase.npy',panda_obj1.export_panda_results)
# del panda_obj1
# panda_obj2 = Panda('Mcase.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = True, remove_missing=True, keep_expression_matrix = False,modeProcess = 'intersection')
# np.save('Panda_Mcase.npy',panda_obj2.export_panda_results)
# del panda_obj2
# panda_obj3 = Panda('Fcont.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = True, remove_missing=True, keep_expression_matrix = False,modeProcess = 'intersection')
# np.save('Panda_Fcont.npy',panda_obj3.export_panda_results)
# del panda_obj3
# panda_obj4 = Panda('Mcont.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = True, remove_missing=True, keep_expression_matrix = False,modeProcess = 'intersection')
# np.save('Panda_Mcont.npy',panda_obj4.export_panda_results)
# del panda_obj4

In [0]:
Panda_Fcase=np.load('drive/My Drive/Panda_Fcase.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
Panda_Mcase=np.load('drive/My Drive/Panda_Mcase.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
Panda_Fcont=np.load('drive/My Drive/Panda_Fcont.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
Panda_Mcont=np.load('drive/My Drive/Panda_Mcont.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])



F case V F cont

In [0]:
# panda_obj3.panda_results = pd.DataFrame(panda_obj3.export_panda_results, columns=['tf','gene','motif','force'])
# subset_panda_results3 = panda_obj3.panda_results.sort_values(by=['force'], ascending=False)
# subset_panda_results3['merge']=subset_panda_results3.tf+'-'+subset_panda_results3.gene

# panda_obj1.panda_results = pd.DataFrame(panda_obj1.export_panda_results, columns=['tf','gene','motif','force'])
# subset_panda_results1 = panda_obj1.panda_results.sort_values(by=['force'], ascending=False)
# subset_panda_results1['merge']=subset_panda_results1.tf+'-'+subset_panda_results1.gene

# www=subset_panda_results3.merge(subset_panda_results1, left_on='merge', right_on='merge')
# www['diff']=www.force_x-www.force_y
# www = www.sort_values(by ='diff' )
# # plt.bar(www['diff'])
# wwww=www[np.abs(www['diff'])>3]
# wwww
# len(pd.unique(wwww['tf_x']))
# import seaborn as sns
# plt.figure(figsize=(12, 30))

# g=sns.barplot(x="diff", y="merge", data=wwww)
# g.set(xlim=(-4.5, 4.5))
# wwww.to_csv('female_caVco.txt')
pd.DataFrame([pd.unique(wwww['tf_x']),pd.unique(wwww['gene_x'])]).to_csv('unqiue_female_caVco.txt',sep='\t',header=None,index=None)

M case V M cont

In [0]:
# panda_obj4.panda_results = pd.DataFrame(panda_obj4.export_panda_results, columns=['tf','gene','motif','force'])
# subset_panda_results4 = panda_obj4.panda_results.sort_values(by=['force'], ascending=False)
# subset_panda_results4['merge']=subset_panda_results4.tf+'-'+subset_panda_results4.gene

# panda_obj2.panda_results = pd.DataFrame(panda_obj2.export_panda_results, columns=['tf','gene','motif','force'])
# subset_panda_results2 = panda_obj2.panda_results.sort_values(by=['force'], ascending=False)
# subset_panda_results2['merge']=subset_panda_results2.tf+'-'+subset_panda_results2.gene

# ww=subset_panda_results4.merge(subset_panda_results2, left_on='merge', right_on='merge')
# ww['diff']=ww.force_x-ww.force_y
# ww = ww.sort_values(by ='diff' )

# zzzz=ww[np.abs(ww['diff'])>3]
# zzzz
# len(pd.unique(zzzz['gene_x']))
# import seaborn as sns
# plt.figure(figsize=(12, 30))

# g=sns.barplot(x="diff", y="merge", data=zzzz)
# g.set(xlim=(-4.5, 4.5))
# zzzz.to_csv('male_caVco.txt')
# [pd.unique(zzzz['tf_x']),pd.unique(zzzz['gene_x'])]
pd.DataFrame([pd.unique(zzzz['tf_x']),pd.unique(zzzz['gene_x'])]).to_csv('unqiue_male_caVco.txt',sep='\t',header=None,index=None)

In [0]:
# sexdiff=wwww.merge(zzzz,left_on='merge', right_on='merge')
plt.figure(figsize=(6, 15))
sexdiff['sexdiff']=sexdiff['diff_x']-sexdiff['diff_y']
sns.barplot(data=sexdiff,x='sexdiff',y='merge')
# g=sns.barplot(x="diff", y="merge", data=wwww)
# g.set(xlim=(-4.5, 4.5))

In [0]:
import networkx as nx

In [0]:
female=nx.from_pandas_edgelist(zzzz, source='tf_x', target='gene_x', edge_attr='diff')
# COPD_GWAS_genes = pd.read_csv('drive/My Drive/Colab Notebooks/COPD_gwas_genes.txt',usecols=[0],header=None)
# pos = nx.draw_kamada_kawai(female)
pos = nx.draw_spectral(female)

nx.draw_networkx(female, pos=pos,mfont_size=4, node_color='blue',node_size=300, font_color='white',edge_color='black')
# COPD_GWAS_genes

In [0]:
male=nx.from_pandas_edgelist(wwww, source='tf_x', target='gene_x', edge_attr='diff')
# COPD_GWAS_genes = pd.read_csv('drive/My Drive/Colab Notebooks/COPD_gwas_genes.txt',usecols=[0],header=None)
pos = nx.draw_spectral(male)
nx.draw_networkx(male, pos=pos,font_size=4, node_color='blue',node_size=300, font_color='white',edge_color='black')
# COPD_GWAS_genes